In [1]:
from typing import Literal
from src.factor.optimizer.util import SolverInput , SolveCond , SolveVars
from src.factor.optimizer.util.solver_input import LinearConstraint , BoundConstraint , CovConstraint , TurnConstraint , ShortConstraint
from src.factor.basic.var import SYMBOL_STOCK_LB , SYMBOL_STOCK_UB

input = SolverInput.rand()




In [2]:
from src.factor.optimizer.solver import MosekSolver , CvxpySolver

In [3]:
msolver = MosekSolver(input)
msolver.solve()

(array([0.2318499 , 0.54128885, 0.22686125]), True, 'optimal')

In [ ]:
class Solver:
    def __init__(self , input : SolverInput , 
                 prob_type : Literal['linprog' , 'quadprog' , 'socp'] = 'socp' ,
                 cvxpy_solver : Literal['mosek','ecos','osqp','scs','clarabel'] = 'mosek' , **kwargs):
        self.input = input
        self.prob_type : Literal['linprog' , 'quadprog' , 'socp'] = prob_type
        self.solver_name = cvxpy_solver.upper()

    def parse_input(self):
        self.alpha    = self.input.alpha
        self.w0       = self.input.w0 if self.input.w0 is not None else np.zeros_like(self.alpha)
        self.wb       = self.input.wb if self.input.wb is not None else np.zeros_like(self.alpha)

        if self.prob_type != 'linprog' and self.input.cov_con and self.input.wb is not None:
            self.cov_con   = self.input.cov_con
        else:
            self.cov_con   = None

        if self.input.turn_con and self.input.w0 is not None:
            self.turn_con  = self.input.turn_con
        else:
            self.turn_con  = None

        if self.input.short_con:
            self.short_con  = self.input.short_con
        else:
            self.short_con  = None

        # variable sequence:
        # num_N , num_T (0 or num_N) , num_S (0 or num_N) , num_L (0 or len(self.F)) , num_Q (0 or 2)
        num_N = len(self.alpha)
        num_T = 0 if not self.conds.turn or not self.turn_con else num_N
        num_S = 0 if not self.conds.short or not self.short_con else num_N
        if (not self.conds.qobj and not self.conds.qcon) or not self.cov_con or self.cov_con.cov_type != 'model':
            num_L = 0
        else: num_L = len(self.cov_con.F)

        self.num_vars = SolveVars(num_N , num_T , num_S , num_L)
        return self

    def solve(self , turn = True , qobj = True , qcon = True , short = True):
        self.conds = SolveCond(turn , qobj , qcon , short)
        self.parse_input()

        x = cp.Variable(self.num_vars.N)
        objective = -self.input.alpha.T @ x
        constraints :list = [
            x <= self.input.bnd_con.ub ,
            x >= self.input.bnd_con.lb ,
        ]

        if self.cov_con:
            if self.num_vars.L:
                l = cp.Variable(self.num_vars.L)
                constraints.append(self.cov_con.F @ (x - self.wb) == l)
                if self.cov_con.lmbd:
                    S_sq = np.sqrt(self.cov_con.S)
                    objective = objective + self.cov_con.lmbd / 2.0 * \
                        (cp.sum_squares(cp.multiply(x - self.wb , S_sq)) + cp.quad_form(l , self.cov_con.C) )
                if self.cov_con.te:
                    constraints.append(cp.sum_squares(cp.multiply(x - self.wb , S_sq)) + 
                                cp.quad_form(l , self.cov_con.C) <= self.cov_con.te ** 2)
            else:
                if self.cov_con.lmbd:
                    objective = objective + self.cov_con.lmbd / 2.0 * cp.quad_form(x , self.cov_con.cov)
                if input.cov_con.te:
                    constraints.append(cp.quad_form(x , self.cov_con.cov) <= self.cov_con.te ** 2)
        
        eq_pos = self.input.lin_con.type == 'fx'
        if np.any(eq_pos):
            mat = self.input.lin_con.A[eq_pos]
            bnd = self.input.lin_con.lb[eq_pos]
            constraints.append(mat @ x == bnd)

        up_pos = np.isin(self.input.lin_con.type,['ra', 'up'])
        lo_pos = np.isin(self.input.lin_con.type,['ra', 'lo'])
        if np.any(up_pos) or np.any(lo_pos):
            mat = np.vstack((self.input.lin_con.A[up_pos], -self.input.lin_con.A[lo_pos]))
            bnd = np.hstack((self.input.lin_con.ub[up_pos], self.input.lin_con.lb[lo_pos]))
            constraints.append(mat @ x <= bnd)

        if self.turn_con and self.num_vars.T:
            t = cp.Variable(self.num_vars.T)
            constraints.append(t >= 0)
            constraints.append(x - t <= self.w0)
            constraints.append(-x - t <= -self.w0)

            if self.turn_con.dbl: constraints.append(cp.sum(t) <= self.turn_con.dbl)
            if self.turn_con.rho: objective = objective + self.turn_con.rho * cp.sum(t)  

        if self.short_con and self.num_vars.S:
            s = cp.Variable(self.num_vars.S)
            constraints.append(s >= 0)
            constraints.append(x - s >= 0)

            if self.short_con.pos:  constraints.append(cp.sum(s) <= self.short_con.pos)
            if self.short_con.cost: objective = objective + self.short_con.cost * cp.sum(s)  

        prob = cp.Problem(cp.Minimize(objective), constraints)
        prob.solve(solver = self.solver_name , **_SOLVER_PARAM.get(self.solver_name , {}))
        status = prob.status
        is_success = (status == 'optimal' or status == 'optimal_inaccurate')
        w = x.value
        return w, is_success, status

In [26]:
import cvxopt

from cvxopt import matrix, solvers , sparse , spdiag
import numpy as np

a = np.eye(10)
b = np.random.rand(5,5)
c = np.zeros((7,7))
d = spdiag([sparse(matrix(a)),sparse(matrix(b)),sparse(matrix(0.,(7,7)))])

In [29]:
sparse(matrix(0.,(7,7)))

<7x7 sparse matrix, tc='d', nnz=0>

In [6]:
b = matrix(a)

In [10]:
sparse(matrix(a))

<10x10 sparse matrix, tc='d', nnz=10>

In [31]:
spdiag(matrix(np.random.rand(5)))

<5x5 sparse matrix, tc='d', nnz=5>

In [37]:
np.vstack([[1,2,],[2,3]])

array([[1, 2],
       [2, 3]])

In [32]:
a = matrix([[1,2,],[2,3]])
print(a)

[ 1  2]
[ 2  3]



In [34]:
a.trans()

<2x2 matrix, tc='i'>

In [1]:
import cvxopt

from cvxopt import matrix, solvers , sparse , spdiag
import numpy as np

import numpy as np

from typing import Literal
from src.factor.optimizer.util import SolverInput , SolveCond , SolveVars
from src.factor.optimizer.util.solver_input import LinearConstraint , BoundConstraint , CovConstraint , TurnConstraint , ShortConstraint
from src.factor.basic.var import SYMBOL_STOCK_LB , SYMBOL_STOCK_UB

inp = SolverInput.rand()

def parse_line_condition(lin_con):
    coef_matrix = lin_con[0]
    bl_bu_bnd_key, bl, bu = lin_con[1][0], lin_con[1][1], lin_con[1][2]
    ineq_bnd_loc = [i for i in range(len(bl_bu_bnd_key)) if bl_bu_bnd_key[i] != "fx"]
    A = coef_matrix[ineq_bnd_loc, :].copy()
    bl_bu_bnds = [bl_bu_bnd_key[ineq_bnd_loc], bl[ineq_bnd_loc], bu[ineq_bnd_loc]]
    #
    fx_bnd_loc = [i for i in range(len(bl_bu_bnd_key)) if bl_bu_bnd_key[i] == "fx"]
    G = coef_matrix[fx_bnd_loc, :].copy()
    h = bl[fx_bnd_loc]
    return A, bl_bu_bnds, G, h

def _solve_with_turn_with_brcov(inp : SolverInput , u, cov_info, wb, lin_con, bnd_con, turn_con, te, solver_params):
    lmbd, F, C, S = cov_info
    w0, to, rho = turn_con
    A, bl_bu_bnds, G, h = parse_line_condition(lin_con)
        mat = np.vstack((self.input.lin_con.A[up_pos], -self.input.lin_con.A[lo_pos]))
        bnd = np.hstack((self.input.lin_con.ub[up_pos], self.input.lin_con.lb[lo_pos]))
        constraints.append(mat @ x <= bnd)

    #
    u_adj, K, A_adj, bl_bu_bnds_adj, G_adj, h_adj, bnd_con_adj = \
        _transform_parameter_with_brcov(u, C, S, F, wb, A, bl_bu_bnds, G, h, bnd_con)
    x0 = (w0 - wb) * np.sqrt(S)
    ineq_matrix, ineq_bnd, eq_matrix, eq_bnd = combine_lin_bnd_condition(A_adj, bl_bu_bnds_adj, G_adj, h_adj, bnd_con_adj)
    #
    N = len(inp.alpha)

    if inp.cov_con.cov_type == 'model':
        L = len(inp.cov_con.F)
        P = spdiag(spdiag(matrix(inp.cov_con.S * inp.cov_con.lmbd)) , 
                   sparse(matrix(0.,(N,N))) ,
                   matrix(inp.cov_con.C * inp.cov_con.lmbd))
    elif inp.cov_con.cov_type == 'normal':
        L = 0
        P = spdiag(matrix(inp.cov_con.S * inp.cov_con.lmbd) , sparse(matrix(0.,(N,N))))
    else:
        L = 0
        P = None
    q = matrix(np.hstack([-inp.alpha, rho * np.ones(N) , np.zeros(L)]))

    cvx_A = np.vstack(
        (np.hstack((eq_matrix, np.zeros((eq_matrix.shape[0], N + L)))),
         np.hstack((K, np.zeros((L, N)), -np.eye(L)))
         ))
    cvx_A = transfer_to_spmatrix(cvx_A)
    cvx_b = matrix(np.hstack((eq_bnd, np.zeros(L))))
    #
    cvx_G = np.vstack(
        (np.hstack((ineq_matrix, np.zeros((ineq_matrix.shape[0], N + L)))),
         np.hstack((np.eye(N), -np.diag(np.sqrt(S)), np.zeros((N, L)))),
         np.hstack((-np.eye(N), -np.diag(np.sqrt(S)), np.zeros((N, L)))),
         np.hstack((np.zeros((N, N)), -np.eye(N), np.zeros((N, L)))),
         np.hstack(([0.0] * N, [1.0] * N, [0.0] * L)).reshape(1, -1),
         np.zeros((1, 2 * N + L)),
         np.diag(np.array([1.0] * N + [0.0] * N + [1.0] * L))
         ))
    cvx_G = transfer_to_spmatrix(cvx_G)
    cvx_h = matrix(np.hstack((ineq_bnd, x0, -x0, np.zeros(N), [to], [te], np.zeros(2 * N + L))))
    dims = {'l': ineq_matrix.shape[0] + N * 3 + 1, 'q': [2 * N + L + 1], 's': []}
    #
    for key, val in solver_params.items():
        solvers.options[key] = val
    sol = solvers.coneqp(P=P, q=q, G=cvx_G, h=cvx_h, dims=dims, A=cvx_A, b=cvx_b)
    #
    status = sol['status']
    if status == 'optimal':
        is_success = True
    else:
        is_success = False
    x = np.array(sol['x'])[:N, 0]
    w = np.array(x) * (1 / np.sqrt(S)) + wb
    return w, is_success, status




def combine_lin_bnd_condition(A, bl_bu_bnds, G, h, bnd_con):
    lb_bu_bnd_key, lb, ub = bnd_con[0], bnd_con[1], bnd_con[2]
    bl_bu_bnd_key, bl, bu = bl_bu_bnds[0], bl_bu_bnds[1], bl_bu_bnds[2]
    bnd_matrix = np.eye(len(lb))
    #
    fx_bnd_loc = [i for i in range(len(lb_bu_bnd_key)) if lb_bu_bnd_key[i] == "fx"]
    if fx_bnd_loc:
        eq_matrix = np.vstack((G, bnd_matrix[fx_bnd_loc, :]))
        eq_bnd = np.hstack((h, lb[fx_bnd_loc]))
    else:
        eq_matrix = G.copy()
        eq_bnd = h.copy()
    #
    up_bl_bu_loc = [i for i in range(len(bl_bu_bnd_key)) if bl_bu_bnd_key[i] in ('ra', 'up')]
    up_lb_ub_loc = [i for i in range(len(lb_bu_bnd_key)) if lb_bu_bnd_key[i] in ('ra', 'up')]
    ineq_matrix = np.vstack((A[up_bl_bu_loc, :], bnd_matrix[up_lb_ub_loc, :]))
    ineq_bnd = np.hstack((bu[up_bl_bu_loc], ub[up_lb_ub_loc]))
    #
    lo_lb_ub_loc = [i for i in range(len(lb_bu_bnd_key)) if lb_bu_bnd_key[i] in ("ra", "lo")]
    lo_bl_bu_loc = [i for i in range(len(bl_bu_bnd_key)) if lb_bu_bnd_key[i] in ("ra", "lo")]
    ineq_matrix = np.vstack((ineq_matrix, -A[lo_bl_bu_loc, :], -bnd_matrix[lo_lb_ub_loc, :]))
    ineq_bnd = np.hstack((ineq_bnd, -bl[lo_bl_bu_loc], -lb[lo_lb_ub_loc]))
    return ineq_matrix, ineq_bnd, eq_matrix, eq_bnd


def transfer_to_spmatrix(x):
    x_loc = np.nonzero(x)
    rtn = spmatrix(x[x_loc], x_loc[0], x_loc[1], size=x.shape)
    return rtn


def _transform_parameter_with_brcov(u, C, S, F, wb, A, bl_bu_bnds, G, h, bnd_con):
    s_vol = np.sqrt(S)
    u_adj = u * (1.0 / s_vol)
    V = np.linalg.cholesky(C).T
    assert np.allclose(V.T.dot(V), C)
    K = V.dot(F) * (1.0 / s_vol)
    G_adj = G * (1.0 / s_vol)
    A_adj = A * (1.0 / s_vol)
    #
    h_adj = h - G.dot(wb)
    #
    bl_bu_bnd_key, bl, bu = bl_bu_bnds[0], bl_bu_bnds[1], bl_bu_bnds[2]
    bl_adj = np.array(bl) - A.dot(wb)
    bu_adj = bu - A.dot(wb)
    bl_bu_bnds_adj = (bl_bu_bnd_key, bl_adj, bu_adj)
    #
    lb_lu_bnd_key, lb, lu = bnd_con[0], bnd_con[1], bnd_con[2]
    lb_adj = (lb - wb) * s_vol
    lu_adj = (lu - wb) * s_vol
    lb_lu_bnds_adj = (lb_lu_bnd_key, lb_adj, lu_adj)
    return u_adj, K, A_adj, bl_bu_bnds_adj, G_adj, h_adj, lb_lu_bnds_adj


def _transform_parameter_with_nmcov(cov, wb, A, bl_bu_bnds, G, h, bnd_con):
    V = np.linalg.cholesky(cov).T
    #
    h_adj = h - G.dot(wb)
    #
    bl_bu_bnd_key, bl, bu = bl_bu_bnds[0], bl_bu_bnds[1], bl_bu_bnds[2]
    bl_adj = np.array(bl) - A.dot(wb)
    bu_adj = bu - A.dot(wb)
    bl_bu_bnds_adj = (bl_bu_bnd_key, bl_adj, bu_adj)
    #
    lb_lu_bnd_key, lb, lu = bnd_con[0], bnd_con[1], bnd_con[2]
    lb_adj = lb - wb
    lu_adj = lu - wb
    lb_lu_bnds_adj = (lb_lu_bnd_key, lb_adj, lu_adj)
    return V, bl_bu_bnds_adj, h_adj, lb_lu_bnds_adj


